In [1]:
# Load libraries
from sklearn.ensemble import AdaBoostClassifier
from sklearn import datasets
# Import train_test_split function
from sklearn.model_selection import train_test_split
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

In [2]:
#loading the dataset with the only columns which has high feature importance(obtained from LGBM)
import numpy as np, pandas as pd
load = ['HasDetections', 'AvSigVersion', 'Census_OSVersion', 'OsBuildLab']
df= pd.read_csv('../input/microsoft-malware-prediction/train.csv',dtype='category',usecols=load)
df['HasDetections'] = df['HasDetections'].astype('int8')

In [3]:
#REPLACING MISSING VALUES IN EACH ROW 
#since majority of the columns are categorical we are replacing the missing values by mode
for column in df.columns:
    df[column].fillna(df[column].mode()[0], inplace=True)

In [4]:
#since the version numbers in our dataset associated with time we are using external dataset provided by the kaggle
from datetime import datetime, date, timedelta

# AS timestamp
datedictAS = np.load('../input/malware-timestamps/AvSigVersionTimestamps.npy',allow_pickle=True)[()]
df['DateAS'] = df['AvSigVersion'].map(datedictAS)  

# OS timestamp
datedictOS = np.load('../input/malware-timestamps-2/OSVersionTimestamps.npy',allow_pickle=True)[()]
df['DateOS'] = df['Census_OSVersion'].map(datedictOS)  

# BL timestamp
def convert(x):
    try:
        d = datetime.strptime(x.split('.')[4],'%y%m%d-%H%M')
    except:
        d = np.nan
    return d
df['DateBL'] = df['OsBuildLab'].map(convert)
df.head()
df.dropna(inplace=True)
print(len(df))
df.isnull().values.any()

8910020


False

In [5]:
#GOOGLE DATA
data = pd.read_csv('../input/google-safe-browsing-transparency-report-data/data.csv')
data['WeekOf'] = data['WeekOf'].map(lambda x: datetime.strptime(x,'%Y-%m-%d').date())
weekdictAS={}
for x in datedictAS: 
    weekdictAS[x] = (datedictAS[x] - timedelta(days= -7+1+datedictAS[x].weekday())).date()
df['WeekOf'] = df['AvSigVersion'].map(weekdictAS)
df = pd.merge(df, data, on='WeekOf', how='left')
data.sample(5)
df.dropna(inplace=True)
print(len(df))
df.isnull().values.any()

8905820


False

In [6]:
#THREAT DATA
data2 = pd.read_csv('../input/malware-avsigversion-threats/AvSigversion_Threats.csv')
cv = pd.DataFrame(data2.groupby('AvSigVersion')['index'].count()).rename({'index':'ThreatCount'},axis=1)
df = pd.merge(df,cv,on='AvSigVersion',how='left')
df['ThreatCount'].fillna(0,inplace=True)
data2.sample(10)
df.dropna(inplace=True)
print(len(df))
df.isnull().values.any()

8905820


False

In [7]:
#FINAL DATA
del df['DateAS'], df['DateOS'], df['DateBL'], df['WeekOf'] 
del df['AvSigVersion'], df['OsBuildLab'], df['Census_OSVersion']
df.dropna(inplace=True)
print(len(df))
df.isnull().values.any()
df.head()

8905820


,HasDetections,Malware sites detected,Phishing sites detected,Malware sites number,Phishing sites number,Attack sites,Compromised sites,Browser warnings,Search warnings,Webmaster response time,Reinfection rate,ThreatCount
0,0,5309.0,27099.0,210850.0,888579.0,205.0,5105.0,8097325.0,13514374.0,69.0,13.0,0.0
1,0,6406.0,30749.0,430100.0,733866.0,273.0,6135.0,13574589.0,32221128.0,60.0,16.0,1.0
2,0,4901.0,27996.0,220571.0,885718.0,211.0,4692.0,6779998.0,14035159.0,66.0,12.0,2.0
3,1,4901.0,27996.0,220571.0,885718.0,211.0,4692.0,6779998.0,14035159.0,66.0,12.0,0.0
4,1,4901.0,27996.0,220571.0,885718.0,211.0,4692.0,6779998.0,14035159.0,66.0,12.0,0.0


In [8]:
#splitting the dataset into test and train
X=df
y=X['HasDetections']
del X['HasDetections']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [10]:
# TRAIN LGBM
import lightgbm as lgb,gc
import pickle
model = lgb.LGBMClassifier(n_estimators=10000, colsample_bytree=0.7, objective='binary', num_leaves=32,
            max_depth=-1, learning_rate=0.1)
h=model.fit(X_train, y_train, eval_metric='auc', eval_set=[(X_test, y_test)], verbose=50,
            early_stopping_rounds=100)
Pkl_Filename = "Pickle_RL_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(h, file)
print("done")

NameError: name 'X_val' is not defined